In [1]:
print("""
FullName : Diallo Mamadou Korka
Studend’s Number: 022028845F
      
Modelling and Analysis of Complex Networks
Assignment : 9 
Number of the dataset : 3
""")


FullName : Diallo Mamadou Korka
Studend’s Number: 022028845F
      
Modelling and Analysis of Complex Networks
Assignment : 9 
Number of the dataset : 3



In [5]:
# import the modules
import urllib.request
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [2]:
#Load the dataset
# Load the Facebook-Ego network
facebook_url = "https://raw.githubusercontent.com/wang422003/Complex-Networks_exercise/main/Datasets/Group3/Facebook-Ego/348.edges"
urllib.request.urlretrieve(facebook_url, "facebook.edges")
facebook_network = nx.read_edgelist("facebook.edges", nodetype=int)

# Load the Twitter-Ego network
twitter_url = "https://raw.githubusercontent.com/wang422003/Complex-Networks_exercise/main/Datasets/Group3/Twitter-Ego/789071.edges"
urllib.request.urlretrieve(twitter_url, "twitter.edges")
twitter_network = nx.read_edgelist("twitter.edges", nodetype=int, create_using=nx.DiGraph())

In [10]:
#G_Facebook = nx.read_edgelist('686.edges')

# Task 1: Randomly delete 10% of the edges
num_edges_to_remove = int(0.1 * facebook_network.number_of_edges())
edges_to_remove = random.sample(facebook_network.edges(), num_edges_to_remove)
facebook_network.remove_edges_from(edges_to_remove)
path_features = facebook_url

/tmp/ipykernel_3605/3002649804.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  edges_to_remove = random.sample(facebook_network.edges(), num_edges_to_remove)


In [11]:
features = pd.read_csv(path_features, sep=" ", header=None)
features.set_index(0, inplace=True)

In [16]:
def feature_similarity(node1, node2):
    if node1 in features.index and node2 in features.index:
        return np.sum(features.loc[node1] == features.loc[node2])
    else:
        return 0  # or any default value


def jaccard_coefficient(node1, node2, graph):
    union_size = len(set(graph.neighbors(node1)) | set(graph.neighbors(node2)))
    if union_size == 0:
        return 0
    intersection_size = len(list(nx.common_neighbors(graph, node1, node2)))
    return intersection_size / union_size

def generate_features(node1, node2, graph):
    return [
        feature_similarity(node1, node2),
        jaccard_coefficient(node1, node2, graph)
    ]

In [17]:
# Task 2: Predict the deleted edges
positive_examples = edges_to_remove  
negative_examples = [(u, v) for u in facebook_network.nodes() for v in facebook_network.nodes() if u < v and not facebook_network.has_edge(u, v)]
negative_examples = random.sample(negative_examples, len(positive_examples))  
examples = positive_examples + negative_examples
labels = [1] * len(positive_examples) + [0] * len(negative_examples)

features_ml = [generate_features(u, v, facebook_network) for u, v in examples]

X_train, X_test, y_train, y_test = train_test_split(features_ml, labels, test_size=0.1, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)


ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects